In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/hivep10
!hdfs dfs -ls /tmp/
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep10/
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep10/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
drwxr-xr-x   - root supergroup          0 2020-02-11 03:54 /tmp/hivep9
Found 5 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 04:06 /tmp/hivep10
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
drwxr-xr-x   - root supergroup          0 2020-02-11 03:54 /tmp/hivep9
copyFromLocal: `tbl0.csv': No such file or directory
copyFromLocal: `tbl1.csv': No such file or directory


In [3]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 17.902 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 2.056 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 3.276 seconds


In [4]:
%%hive
DROP TABLE IF EXISTS pt10;
CREATE TABLE pt10
AS
SELECT letra, COUNT(1)
FROM t0 LATERAL VIEW explode(c3) adTable AS letra,numero
GROUP BY letra;

INSERT OVERWRITE DIRECTORY '/tmp/hivep10'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM pt10;

DROP TABLE IF EXISTS pt10;
OK
Time taken: 0.042 seconds
CREATE TABLE pt10
AS
SELECT letra, COUNT(1)
FROM t0 LATERAL VIEW explode(c3) adTable AS letra,numero
GROUP BY letra;
Query ID = root_20200211040853_fff94ffe-a3b1-4ecc-a2bc-f144a28370bc
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1581390426457_0007, Tracking URL = http://714a45eef6db:8088/proxy/application_1581390426457_0007/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581390426457_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-11 04:09:09,210 Stage-1 map = 0%,  reduce = 0%
2020-02-11 04:09:17,564 Stage-1

In [5]:
!hdfs dfs -cat /tmp/hivep10/000000_0 

aaa,13
bbb,16
ccc,23
ddd,23
eee,15
fff,20
ggg,13
hhh,16
iii,18
jjj,18


In [6]:
!hdfs dfs -copyToLocal /tmp/hivep10/000000_0 output;

In [7]:
%%hive
DROP TABLE pto10;
DROP TABLE t0;

DROP TABLE pto10;
OK
Time taken: 0.283 seconds
DROP TABLE t0;
OK
Time taken: 0.805 seconds


In [8]:
!rm *.tsv *.log

rm: cannot remove 'derby.log': Text file busy


In [9]:
%hive_quit